In [2]:
%load_ext autoreload
%autoreload 2

import duckdb
import json
import os
import pandas as pd
import pathlib

from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Initialize DuckDB connection
conn = duckdb.connect("Piper.db")

### IUCN (integrando via API)
#### Adicionei informações de extant e land_regions

In [77]:
conn.execute("ALTER TABLE iucn ADD COLUMN extant BOOLEAN DEFAULT FALSE")
conn.execute("ALTER TABLE iucn ADD COLUMN land_regions TEXT")

In [78]:
res = conn.execute("SELECT * FROM iucn").df()
res.head()

,internalTaxonId,scientificName_x,kingdomName,phylumName,className,orderName,familyName,genusName,speciesName,infraType,...,useTrade,systems,conservationActions,realm,yearLastSeen,possiblyExtinct,possiblyExtinctInTheWild,scopes,extant,land_regions
0,45862,Piper skutchii,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,skutchii,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,False,None
1,45861,Piper schuppii,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,schuppii,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,False,None
2,45863,Piper sodiroi,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,sodiroi,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,False,None
3,45864,Piper stipulosum,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,stipulosum,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,False,None
4,45814,Piper achupallasense,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,achupallasense,NaN,...,None,Terrestrial,Not confirmed in any Ecuadorean protected area.,Neotropical,NaN,False,False,Global,False,None


In [79]:
conn.execute("DROP TABLE IF EXISTS iucn_complete")
conn.execute("CREATE TABLE iucn_complete AS SELECT * FROM iucn LIMIT 0;")

In [80]:
for _, row in res.iterrows():
    url = "https://api.iucnredlist.org/api/v4/assessment/%d" % row["assessmentId"]
    response = make_requests(url, headers={"Authorization": os.environ["IUCN_API"]})
    if response and "locations" in response:
        for l in response["locations"]:
            extant = True if l["presence"] == 'Extant' else False
            land_regions = l["description"]["en"].replace("'", "''")
            print(row["assessmentId"], extant, land_regions)
            sql = f"""
            INSERT INTO iucn_complete
            VALUES ({row['internalTaxonId']}, '{row['scientificName_x']}', '{row['kingdomName']}', '{row['phylumName']}', '{row['className']}', '{row['orderName']}', '{row['familyName']}', '{row['genusName']}', '{row['speciesName']}', '{row['infraType']}', '{row['infraName']}', '{row['infraAuthority']}', '{row['subpopulationName']}', '{row['authority']}', '{row['taxonomicNotes']}', {row['assessmentId']}, '{row['scientificName_y']}', '{row['redlistCategory']}', '{row['redlistCriteria']}', {row['yearPublished']}, '{row['assessmentDate']}', '{row['criteriaVersion']}', '{row['language']}', '{None}', '{None}', '{None}', '{row['population']}', '{row['populationTrend']}', '{None}', '{None}', '{row['systems']}', '{None}', '{row['realm']}', '{row['yearLastSeen']}', {row['possiblyExtinct']}, {row['possiblyExtinctInTheWild']}, '{row['scopes']}', {extant}, '{land_regions}')
            """
            print(sql)
            conn.execute(sql)

11015485 True Ecuador

            INSERT INTO iucn_complete
            VALUES (45862, 'Piper skutchii', 'PLANTAE', 'TRACHEOPHYTA', 'MAGNOLIOPSIDA', 'PIPERALES', 'PIPERACEAE', 'Piper', 'skutchii', 'nan', 'nan', 'nan', 'nan', 'Trel. &amp; Yunck.', 'nan', 11015485, 'Piper skutchii', 'Endangered', 'B1ab(iii)', 2004, '2004-04-30 00:00:00 UTC', '3.1', 'English', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'Terrestrial', 'None', 'Neotropical', 'nan', False, False, 'Global', True, 'Ecuador')
            
11015569 True Ecuador

            INSERT INTO iucn_complete
            VALUES (45861, 'Piper schuppii', 'PLANTAE', 'TRACHEOPHYTA', 'MAGNOLIOPSIDA', 'PIPERALES', 'PIPERACEAE', 'Piper', 'schuppii', 'nan', 'nan', 'nan', 'nan', 'A.H. Gentry', 'nan', 11015569, 'Piper schuppii', 'Vulnerable', 'D2', 2004, '2004-04-30 00:00:00 UTC', '3.1', 'English', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'Terrestrial', 'None', 'Neotropical', 'nan', False, False, 'Global', True, 'Ecuad

In [81]:
conn.close()